In [2]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt
from idaes.apps.uncertainty_propagation.sens import get_dsdp

## Define fixed bed model 

In [3]:
from fixed_bed_model_indexed_dynamic import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:228)


In [4]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Argument:
        timepoints: the timesteps
        time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    
    add_equations(doe)
    
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
    
    
def initialize(doe, init_point):
    '''
    Argument: 
        init_point: initial point, csv file
    '''
    # Initialize 
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp(feed, bath, y):
    dv_dict_overall = {'temp_feed': {0: feed}, 'temp_bath': {0:bath}, 'yfeed': {0: y}}
    return dv_dict_overall

In [5]:
NFEt = 68
time_scale = 3200
collo = False
exp_time_ = time_points(NFEt, time_scale)

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#initialize_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv'
initialize_point = './priors/20210916_feed313_bath293.csv'
#init_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/fixed_bed_initial/T353y20_5e3.csv'
#initialize_point = './priors/20210916_feed313_bath293_5e3.csv'
#init_point = 'init_test.csv'
#init_point = 'T353y20_5e3.csv'

In [6]:
def model_integrate(scena, args_list, exp_time=exp_time_, init_point=initialize_point):
    test = create_model(scena, temp_feed=293.15, temp_bath=293.15, y=0.4, doe_model=args_list[0], k_aug=args_list[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


In [7]:
# choose from model and k_aug 
model_opt = 'doe'

if (model_opt == 'doe'):
    args_ = [True, False]
    
elif (model_opt =='kaug'):
    args_ = [True, True]
    

parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#dv_pass = {'temp_feed': [0], 'temp_bath':[0], 'yfeed': [0]}
#dv_pass = {'temp_feed': None, 'temp_bath':None , 'yfeed': None}
dv_pass = {'temp_feed': None, 'yfeed': None}

measurement = ['FCO2', 'temp']
measurement_extra = {'FCO2': [19], 'temp': [10, 19]}

t_measure = exp_time_


In [8]:
# empty prior
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0.]
 [0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0.]
 [0. 1.]]


In [9]:
fim_init = [[0.0029331 , 0.01236752],
 [0.01236752 , 0.71783208]]

#f2 = open('jac_initial', 'rb')
#jac_init = pickle.load(f2)
#f2.close()

jac_init = jac_init_1.copy()
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

NameError: name 'jac_init_1' is not defined

In [ ]:
import os

scores = {} # scores is an empty dict already
target='jac_initial'
if os.path.getsize(target) > 0:      
    with open(target, "rb") as f:
        unpickler = pickle.Unpickler(f)
        # if file is not empty scores will be equal
        # to the value unpickled
        scores = unpickler.load()
        
print(scores)

## Compute FIM

In [ ]:
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
##sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

# feed, bath, y
exp1 = generate_exp(293.15, 293.15, 0.4)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, 
                                store_output = 'store_output', read_output='store_output_fco2',
                                formula='central')

if sensi_opt == 'simultaneous_finite':
    result.extract_FIM(doe_object.m, doe_object.design_timeset,
                                                          doe_object.square_result, doe_object.obj_opt)
else: 
    result.calculate_FIM(doe_object.jac, doe_object.design_values)

In [ ]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

In [ ]:
print(doe_object.dsdp)

In [ ]:
import pickle

with open('store_output_botht', 'rb') as f:
    data = pickle.load(f)
    f.close()
print(data)

In [ ]:
doe_object.models[0]

In [ ]:
breakthrough_modify2(doe_object.models[2])

In [ ]:
def breakthrough_modify2(m, file=None, source="lab"):
    '''
    Draw breakthrough curve of the Pyomo model
    
    Arguments:
        m: Pyomo model
        file: when source = 'computer', this is where the computer experimental data is stored. Otherwise, its default is None
        source: if computer, it is plotting the computer experiment data. If lab, its comparing the experiments from the lab
    
    Return: None 
    
    Other: plot
    
    '''
    
    
    # unit: mol/m3
    outlet_den, _, _, _ = extract3d(m, m.C, 'CO2')
    
    # unit: cm/s
    outlet_vel, _, _, _ = extract2d(m, m.v)
    
    outlet_n2, _, _, _ = extract3d(m, m.C, 'N2')
    
    model_temp, _,_,_ = extract2d(m,m.temp)
    
    
    T = []
    for j in m.t:
        T.append(value(j))
        t_final = T[-1]
        
    
    if source == "lab":
        break_wvu = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/breakthrough_wvu.csv')
    
        exp = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/co2_breakthrough.csv')
        
        data_c1 = exp['FCO2']
        data_c = np.asarray(data_c1)
        data_t1 = exp['time']  
        data_t = np.asarray(data_t1*60+10)
        new_data = np.interp(T, data_t, data_c)
    
        fco2 = []
        for t in m.t:
            fco2.append(value(m.FCO2[0,19,t]))
        
        yco2 = []
        for i in range(len(fco2)):
            yco2.append(fco2[i]/fco2[-1])
            
        # real value of FCO2. for verification
        comp = outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1])
    
        plt.plot(exp['time']*60, exp['FCO2'],'b.', color='r', label='Experimental data')
        #plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.plot(T, yco2, label='Model prediction')
        
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
    elif source == "computer": 
        
        sol = pd.read_csv(file)
    
        #for i in range(len(time_exp)):    
        #    yco2_exp[time_exp[i]] = sol['FCO2'][i]
        #    temp_mid_exp[time_exp[i]] = sol['temp_mid'][i]
        #    temp_end_exp[time_exp[i]] = sol['temp_end'][i]
        
        exp_fco2 = np.asarray(sol['FCO2'].values.tolist())
        exp_temp_mid = np.asarray(sol['temp_mid'].values.tolist())
        exp_temp_end = np.asarray(sol['temp_end'].values.tolist())
        
        plt.plot(T, exp_fco2/exp_fco2[-1], label = 'Experimental data')
        plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
        
        plt.plot(T, exp_temp_mid, label='Experimental data of middle T')
        plt.plot(T, model_temp[10,:], label='Model prediction of middle T')
        plt.plot(T, exp_temp_end, label='Experimental data of end T')
        plt.plot(T, model_temp[19,:], label='Model prediction of end T')
        plt.xlabel('time [s]')
        plt.ylabel('Temperature [K]')
        plt.title('Temperature model prediction and experimental data')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()

## Gradient-based optimization 

In [10]:
# feed, bath, y
exp1 = generate_exp(313.15, 293.15, 0.15)



### Sequential initial solution 

In [12]:
sensi_opt = 'sequential_finite'

doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, 
                                store_output = 'store_output', read_output=None,
                                formula='central')

result.calculate_FIM(doe_object.jac, doe_object.design_values)


Sensitivity information is scaled by its corresponding parameter nominal value.
This scenario: {'fitted_transport_coefficient': {0: 214.12}, 'ua': {0: 15.424948470398375}, 'jac-index': {'fitted_transport_coefficient': [0], 'ua': [0]}, 'eps-abs': {'fitted_transport_coefficient': 2.1412, 'ua': 0.15424948470398375}, 'scena-name': [0]}
The inlet feed density is 41.0275814575221 [mol/m3]
the number of timepoints is 68
The init point is ./priors/20210916_feed313_bath293.csv
Model # of time grid is 69 , initial point # of time grid is 69
temp_feed is fixed at  313.15
yfeed is fixed at  0.15
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from sou

 189r 0.0000000e+00 1.41e-03 3.51e+02  -5.0 2.92e-02    -  1.93e-02 2.36e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 1.27e-03 3.24e+02  -5.0 2.91e-02    -  3.19e-02 1.04e-01f  1
 191r 0.0000000e+00 1.14e-03 2.96e+02  -5.0 2.89e-02    -  6.73e-02 1.03e-01f  1
 192r 0.0000000e+00 1.12e-03 2.91e+02  -5.0 2.77e-02    -  7.31e-02 2.24e-02f  1
 193r 0.0000000e+00 1.03e-03 2.66e+02  -5.0 2.69e-02    -  2.31e-02 8.62e-02f  1
 194r 0.0000000e+00 1.02e-03 2.62e+02  -5.0 2.27e-02    -  4.00e-02 4.06e-03f  1
 195r 0.0000000e+00 9.79e-04 2.52e+02  -5.0 1.90e-02    -  3.47e-02 4.32e-02f  1
 196r 0.0000000e+00 9.66e-04 4.22e+02  -5.0 1.88e-02    -  8.90e-02 1.32e-02f  1
 197r 0.0000000e+00 9.30e-04 4.06e+02  -5.0 1.90e-02    -  4.07e-02 3.76e-02f  1
 198r 0.0000000e+00 9.09e-04 3.89e+02  -5.0 1.65e-02    -  4.38e-03 2.31e-02f  1
 199r 0.0000000e+00 9.00e-04 3.91e+02  -5.0 1.65e-02    -  3.15e-02 9.72e-03f  1
iter    objective    inf_pr 

the number of timepoints is 68
The init point is ./priors/20210916_feed313_bath293.csv
Model # of time grid is 69 , initial point # of time grid is 69
temp_feed is fixed at  313.15
yfeed is fixed at  0.15
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All tec

Output this time:  [0.000199428298248, 4.15897677145134e-05, 4.18875640037504e-05, 4.240081547472943e-05, 4.285536903136261e-05, 4.3253654478415494e-05, 4.3610011184206695e-05, 4.392602313007304e-05, 4.4221084671629736e-05, 4.449824881723668e-05, 4.478729399562983e-05, 4.5102014114971564e-05, 4.5498493722466465e-05, 4.602352080996476e-05, 4.679751463216151e-05, 4.796408183352939e-05, 4.978207989993139e-05, 5.268810550634699e-05, 5.718589938173251e-05, 6.457307226767105e-05, 7.56758961667779e-05, 9.400835698714351e-05, 0.00012064927061089876, 0.00016362492032851207, 0.00022431895705843217, 0.0003169261684327918, 0.0004461940059715864, 0.0006274650105156253, 0.0008843573815780081, 0.0012099570868426466, 0.0016884424187817526, 0.0022396700652917356, 0.0030730274090683468, 0.003976475206209571, 0.005325445816718029, 0.006786844244208956, 0.008805524053321526, 0.01117025121467202, 0.014006122631551737, 0.018429882880389013, 0.022760902442363118, 0.026857718442132614, 0.030686234002280384, 0

Output this time:  [0.000199428298248, 4.156079776071428e-05, 4.186773815329995e-05, 4.2386058726678675e-05, 4.2845086899769506e-05, 4.324631597604595e-05, 4.3604997794626435e-05, 4.392195618071767e-05, 4.4216772557671385e-05, 4.449192990913498e-05, 4.4776188258637974e-05, 4.508279218725344e-05, 4.546418260400839e-05, 4.5966399570083096e-05, 4.6700446000677244e-05, 4.7808383297473426e-05, 4.952654540999051e-05, 5.228707422734509e-05, 5.654913020678204e-05, 6.35865614242402e-05, 7.417176838994665e-05, 9.170366635986689e-05, 0.00011730731790815093, 0.0001585907702813316, 0.00021743721516521912, 0.0003068132253206722, 0.00043318845709654604, 0.000608981812875936, 0.0008617458306216008, 0.0011795627626253855, 0.0016518853859392307, 0.0021946802586777017, 0.0030171224022230796, 0.003915324152162183, 0.005243160812283472, 0.006709339281233267, 0.008688747014108374, 0.011077602526129167, 0.013846889581147466, 0.018277745110602202, 0.02263825105988997, 0.026764280603760862, 0.03062009659192202

Output this time:  [0.000199428298248, 4.158983497112246e-05, 4.188756432950011e-05, 4.240081778575322e-05, 4.285536904159594e-05, 4.325365826507481e-05, 4.361001544889406e-05, 4.392603360019573e-05, 4.422110319991293e-05, 4.449828230644658e-05, 4.4787355653911307e-05, 4.510211751585779e-05, 4.549868008398653e-05, 4.6023823871184175e-05, 4.6798050528150947e-05, 4.7964936489890325e-05, 4.978356291089551e-05, 5.269044301774001e-05, 5.7189822914627274e-05, 6.457924805099343e-05, 7.568572031473033e-05, 9.402393670956967e-05, 0.00012067245171466352, 0.00016366189704138643, 0.00022437009178602553, 0.00031700736677380074, 0.00044629869962392913, 0.0006276272508997101, 0.0008845564333794297, 0.0012102459866769966, 0.0016887962544383393, 0.002240128297916874, 0.0030736238234987967, 0.003977143805286009, 0.005326412248318876, 0.00678776695988081, 0.008807025428847605, 0.011171485781317814, 0.014008363203863037, 0.018432199594661795, 0.022762944511483828, 0.026859431412007553, 0.03068760051643097

In [15]:
print(doe_object.jac)

jac_init = np.zeros((len()))

for i, iname in enumerate(measurement):
    for j in parameter:
        for t in measurement_time:
            jac_init[i,j,t] = doe_object.jac[iname][]

{'fitted_transport_coefficient': [0.0, 2.8844226902382066e-06, 1.972284594093816e-06, 1.4692339188211034e-06, 1.0243074836814022e-06, 7.335210246830007e-07, 5.028803186060459e-07, 4.138864732879208e-07, 4.4305060999107415e-07, 6.531428358011113e-07, 1.1495715644206287e-06, 1.9919071219216204e-06, 3.5430255115774864e-06, 5.892196791356584e-06, 1.0006555005953019e-05, 1.6024689017808975e-05, 2.629651723535271e-05, 4.118111107612758e-05, 6.54329862743312e-05, 0.00010108156442178079, 0.00015423096038908913, 0.00023552733189825386, 0.00034150935769847776, 0.0005128127121254693, 0.0007003097538498707, 0.0010270300377922183, 0.0013180717552757966, 0.0018740817123721874, 0.0022836732649691763, 0.0030831050244101025, 0.0036818204378713153, 0.004563020023369189, 0.005615918778122575, 0.0061887937398191675, 0.008249454604830403, 0.007816694732036252, 0.011713057274706331, 0.009300687815675009, 0.016133077837293414, 0.015233767846124095, 0.01228448753794256, 0.009365139521896881, 0.006636785017630

In [13]:
fim_init = result.FIM

#jac_init = jac_init_1.copy()
    
l_init = np.linalg.cholesky(fim_init)
print(l_init)

[[0.0636188  0.        ]
 [0.38310515 1.14638156]]


In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

square_result, optimize_result = doe_object.optimize_doe(exp1, if_optimize=True, if_Cholesky=True, 
                                                         scale_nominal_param_value=True, objective_option='det', 
                                                         L_initial=l_init, jac_initial=jac_init, fim_initial=fim_init)

In [ ]:
jac_init_1 = doe_object.analysis_square.jac_extracted.copy()
print(type(jac_init_1))
print(jac_init_1[0,0,0])

print(jac_init_1)

fo = open('jac_initial', 'wb')
pickle.dump(jac_init_1, fo)
fo.close



In [ ]:
print('======Result summary======')
print('This optimization is solved with status:', optimize_result.status)
print('It gives solution:', optimize_result.solution)
print('The log10(OF) optimized is:', optimize_result.obj_value)
print('The result FIM is:', optimize_result.FIM)

t_list = []
for t in optimize_result.model.t:
    t_list.append(t)

T_list = []
for i in t_list:
    T_list.append(value(optimize_result.model.T[i]))

si=16
plt.rc('axes', titlesize=si)
plt.rc('axes', labelsize=si)
plt.rc('xtick', labelsize=si)
plt.rc('ytick', labelsize=si)
plt.rc('legend', fontsize=12)
    
plt.plot(t_list, T_list, 'b', linewidth=2)
#plt.scatter(t_list, T_list, 'b')
plt.ylabel('T [$K$]')
plt.xlabel('Time [$h$]')
plt.show()

## Heatmap

In [ ]:
#design_ranges = [list(np.linspace(293,333,5)), list(np.linspace(0.1,0.3,6))]
design_ranges = [[293, 303, 313, 323, 333], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

dv_apply_name = ['temp_feed','yfeed']

dv_apply_time = [[0],[0]]

# Define experiments
exp1 = generate_exp(313.15, 303.15, 0.15)
    
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
#sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

print(design_ranges)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_,
                                 measurement_extra_index = measurement_extra)

all_fim = doe_object.run_grid_search(exp1, design_ranges,dv_apply_name, dv_apply_time, 
                                     mode=sensi_opt, tee_option=False,
                                    scale_nominal_param_value=True, 
                                     store_name='20210919_heatmap', read_name='20210919_heatmap'
                                    )



In [ ]:
test = all_fim.extract_criteria()

fixed = {}
all_fim.figure_drawing(fixed, ['temp_feed','yfeed'], 'Fixed bed','$feed T [K]$', 'y' )

In [ ]:
na = ['temp_feed', 'yfeed']
dd = ['temp_feed', 'yfeed']
a = [[293.0, 303.0, 313.0, 323.0, 333.0], [0.1, 0.14, 0.18, 0.22, 0.26, 0.3]]

In [ ]:
print(a[na.index('temp_feed')])